In [1]:
from datetime import date, timedelta
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm, tnrange

from sklearn.metrics import mean_squared_error
import lightgbm as lgb

import mlflow
import mlflow.sklearn

from config import (
    RAW_DATA_DIR,
    FEATURE_DIR,
    LAG_DICT,
    SLIDING_DICT
)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
# solve lightgbm error on MAC
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# load data
df_train = pd.read_csv(
    RAW_DATA_DIR+'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    RAW_DATA_DIR+'test.csv', usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    RAW_DATA_DIR+'items.csv',
).set_index("item_nbr")

stores = pd.read_csv(
    RAW_DATA_DIR+'stores.csv',
).set_index("store_nbr")

transactions_df = pd.read_csv(
    RAW_DATA_DIR+'transactions.csv'
)

### Test Period

2017-08-16 to 2017-08-31

In [4]:
test_start = date(2017, 8, 16)
test_end = date(2017,8, 31)

In [5]:
valid_start = test_start - timedelta(16)
while(1):
    if valid_start.weekday() == test_start.weekday():
        break
    valid_start = valid_start-timedelta(days=1)
valid_end = valid_start + timedelta(15)
print('valid starts from {} to {}'.format(valid_start, valid_end))

valid starts from 2017-07-26 to 2017-08-10


### Valid Period

Considering the more nearer peiods of sales data may have more in common, it would be better to find the nearest period as valid period.

Based on the analysis before, we assume the sales data is periodically with the frequency of 7 days, so we want to keep that feature same
in the train, valid and test period.

So finally, we choose valid period:

2017-07-26 to 2017-08-10


In [6]:
valid_start = date(2017, 7, 26)
valid_end = date(2017, 8, 10)

### Filter Period

#### Earthquake happended on April 16, 2016. It may affect for the next several weeks.

In [7]:
# filter the period which is affected by earthquake.
filter_date = date(2016,4,16) + timedelta(7*4)
lag_max = 140
train_start=  filter_date+timedelta(days=lag_max)

while(1):
    train_start = train_start + timedelta(1)
    if train_start.weekday() == valid_start.weekday():
        break
print('train datasets starts from {}'.format(train_start))

train datasets starts from 2016-10-05


## Training Period

In [8]:
train_start = date(2017,2,8)

### Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.


In [9]:
df_train = df_train[df_train['date']>=filter_date]

/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


#### Promo feature

In [10]:
promo_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]]

# missing onpromotions filling
promo_train = promo_train.unstack(level=-1).fillna(False)
promo_train.columns = promo_train.columns.get_level_values(1)

In [11]:
# missing test onpromotions filling
promo_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_test.columns = promo_test.columns.get_level_values(1)
# filter those items/stores in promo_test but not in promo_train
promo_test = promo_test.reindex(promo_train.index).fillna(False)

In [12]:
promo_features = pd.concat([promo_train, promo_test], axis=1)
del promo_test, promo_train

In [13]:
transactions_df = pd.read_csv(
    RAW_DATA_DIR+'transactions.csv'
)

## Label

In [23]:
# label
df_train = df_train.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)
# tmp = df_train.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)

## Transactions

In [64]:
transactions_df['date'] = pd.to_datetime(transactions_df['date'])
transactions_df = transactions_df.set_index(["store_nbr", "date"])[["transactions"]].unstack(level=-1).fillna(0)
transactions_df = transactions_df.reindex(df_train.index.get_level_values(0))
transactions_df.columns = transactions_df.columns.get_level_values(1)

## Item

In [43]:
items = items.reindex(df_train.index.get_level_values(1))

#### Item Family

In [45]:
items['family'] = items['family'].astype('category')
item_family_features = items.family.cat.codes.values

#### Item's class

In [46]:
items['class'] = items['class'].astype('category')
item_class_features = items['class'].cat.codes.values

## Store

In [47]:
stores = stores.reindex(df_train.index.get_level_values(0))

#### Store's city

In [48]:
stores['city'] = stores['city'].astype('category')
store_city_features = stores['city'].cat.codes.values

#### Store's state

In [49]:
stores['state'] = stores['state'].astype('category')
store_state_features = stores['state'].cat.codes.values

#### Store's type

In [50]:
stores['type'] = stores['type'].astype('category')
store_type_features = stores['type'].cat.codes.values

#### Store's cluster

In [51]:
stores['cluster'] = stores['cluster'].astype('category')
store_cluster_features = stores['cluster'].cat.codes.values

In [52]:
df_train.columns = df_train.columns.get_level_values(1)

#### Filling missing date

In [53]:
date_list = df_train.columns
obj_list = pd.date_range(filter_date, test_start-timedelta(1))
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    df_train[i] = 0

2016-12-25 00:00:00


In [54]:
date_list = promo_features.columns
obj_list = pd.date_range(filter_date, test_end)
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    promo_features[i] = 0

2016-12-25 00:00:00


#### Lagging and sliding windows

In [55]:
LAG_DICT = {'unit_sales': [1,2,3,4,5,6,7,14,21,28,35,42,49,56,63],
            'onpromotion': [2, 3,4,5,6, 7, 14, 21],
            'transactions': [1, 2, 3, 4, 5, 6, 7, 14, 21]}

SLIDING_DICT = {'unit_sales': [3, 4, 5, 6, 7, 14, 21, 30, 60, 63]}



# initialise dirs
RAW_DATA_DIR = 'datasets/'

In [57]:
def get_timespan(df, 
                 start_time,
                 minus,
                 periods,
                 freq='D'):
    return df[pd.date_range(start_time - timedelta(days=minus), periods=periods, freq=freq)]

def gen_dataset(df,
                promo_features,
                item_family_features,
                item_class_features,
                store_city_features,
                store_state_features,
                store_type_features,
                store_cluster_features,
                transactions_df,
                start_time,
                is_train=True):
    # init
    X = pd.DataFrame()

    for i in LAG_DICT['unit_sales']:
        X['lag_{}_sales'.format(i)] = get_timespan(df, start_time, i, 1).values.ravel()
    
    for i in LAG_DICT['onpromotion']:
        X['sum_{}_promo'.format(i)] = get_timespan(promo_features, start_time, i, 1).sum(axis=1).ravel()

    for i in SLIDING_DICT['unit_sales']:
        X["mean_{}_sales".format(i)] = get_timespan(df, start_time, i, i).mean(axis=1).values
        X["std_{}_sales".format(i)] = get_timespan(df, start_time, i, i).std(axis=1).values

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, start_time, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, start_time, 140-i, 20, freq='7D').mean(axis=1).values

    for i in LAG_DICT['transactions']:
        X['lag_{}_transactions'.format(i)] = get_timespan(transactions_df, start_time, i, 1).values.ravel()

    # for the next to-predict 16 days 
    for i in range(16):
        X["promo_{}".format(i)] = promo_features[start_time + timedelta(days=i)].values.astype(np.uint8)

    X['item_family_features'] = item_family_features

    X['item_class_features'] = item_class_features

    X['store_city_features'] = store_city_features

    X['store_state_features'] = store_state_features

    X['store_type_features'] = store_type_features

    X['store_cluster_features'] = store_cluster_features
        
    if is_train:
        y = df[pd.date_range(start_time, periods=16)].values
        return X, y
    return X


#### Generate train, valid and test sets

In [76]:
print("Preparing dataset...")

nbr_weeks = int((valid_start - train_start).days/7)

X_l, y_l = [], []

for i in tqdm(range(nbr_weeks), desc = 'No. of week'):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = gen_dataset(
        df_train,
        promo_features,
        item_family_features,
        item_class_features,
        store_city_features,
        store_state_features,
        store_type_features,
        store_cluster_features,
        transactions_df,
        train_start + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
#     break

No. of week:   0%|          | 0/24 [00:00<?, ?it/s]

Preparing dataset...


No. of week: 100%|██████████| 24/24 [00:21<00:00,  1.11it/s]


In [77]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

In [79]:
X_val, y_val = gen_dataset(df_train,
                           promo_features,
                           item_family_features,
                           item_class_features,
                           store_city_features,
                           store_state_features,
                           store_type_features,
                           store_cluster_features,
                           transactions_df,
                           valid_start)
X_test = gen_dataset(df_train, 
                    promo_features,
                    item_family_features,
                    item_class_features,
                    store_city_features,
                    store_state_features,
                    store_type_features,
                    store_cluster_features,
                    transactions_df,
                    test_start, is_train=False)

#### Train Model

In [80]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 200
val_pred = []
test_pred = []
cate_vars = ['item_family_features',
            'item_class_features',
            'store_city_features',
            'store_state_features',
            'store_type_features',
            'store_cluster_features']

Training and predicting models...


In [81]:
for i in tqdm(range(16)):
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * nbr_weeks) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)

    bst = lgb.train(
        params,
        dtrain,
        num_boost_round=MAX_ROUNDS,
#         verbose_eval = False,
        valid_sets=[dtrain, dval], early_stopping_rounds=50)
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
    


  0%|          | 0/16 [00:00<?, ?it/s]/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	training's l2: 1.05102	valid_1's l2: 1.00592
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.981373	valid_1's l2: 0.938532
[3]	training's l2: 0.919894	valid_1's l2: 0.879283
[4]	training's l2: 0.864084	valid_1's l2: 0.825476
[5]	training's l2: 0.813703	valid_1's l2: 0.776978
[6]	training's l2: 0.766517	valid_1's l2: 0.731337
[7]	training's l2: 0.723585	valid_1's l2: 0.690035
[8]	training's l2: 0.684655	valid_1's l2: 0.652411
[9]	training's l2: 0.650701	valid_1's l2: 0.619872
[10]	training's l2: 0.618776	valid_1's l2: 0.589183
[11]	training's l2: 0.589819	valid_1's l2: 0.561363
[12]	training's l2: 0.563715	valid_1's l2: 0.536369
[13]	training's l2: 0.539947	valid_1's l2: 0.513612
[14]	training's l2: 0.519278	valid_1's l2: 0.493935
[15]	training's l2: 0.499749	valid_1's l2: 0.475246
[16]	training's l2: 0.482751	valid_1's l2: 0.459138
[17]	training's l2: 0.467343	valid_1's l2: 0.444508
[18]	training's l2: 0.453455	valid_1's l2: 0.431288
[19]	training

  6%|▋         | 1/16 [01:45<26:24, 105.60s/it]

[1]	training's l2: 0.94861	valid_1's l2: 0.92514
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.890743	valid_1's l2: 0.868577
[3]	training's l2: 0.838411	valid_1's l2: 0.817683
[4]	training's l2: 0.791094	valid_1's l2: 0.771371
[5]	training's l2: 0.748662	valid_1's l2: 0.730161
[6]	training's l2: 0.710467	valid_1's l2: 0.693095
[7]	training's l2: 0.675701	valid_1's l2: 0.659396
[8]	training's l2: 0.643618	valid_1's l2: 0.628102
[9]	training's l2: 0.614662	valid_1's l2: 0.599758
[10]	training's l2: 0.588515	valid_1's l2: 0.574308
[11]	training's l2: 0.564743	valid_1's l2: 0.551102
[12]	training's l2: 0.543097	valid_1's l2: 0.530072
[13]	training's l2: 0.523631	valid_1's l2: 0.511141
[14]	training's l2: 0.505973	valid_1's l2: 0.493935
[15]	training's l2: 0.489997	valid_1's l2: 0.47848
[16]	training's l2: 0.475799	valid_1's l2: 0.464806
[17]	training's l2: 0.462933	valid_1's l2: 0.452526
[18]	training's l2: 0.451015	valid_1's l2: 0.440911
[19]	training'

 12%|█▎        | 2/16 [03:26<24:17, 104.12s/it]

...

 88%|████████▊ | 14/16 [23:23<03:22, 101.06s/it]

[1]	training's l2: 1.05906	valid_1's l2: 1.0086
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.994618	valid_1's l2: 0.946203
[3]	training's l2: 0.936245	valid_1's l2: 0.889772
[4]	training's l2: 0.885967	valid_1's l2: 0.840585
[5]	training's l2: 0.838041	valid_1's l2: 0.79409
[6]	training's l2: 0.796889	valid_1's l2: 0.753991
[7]	training's l2: 0.757313	valid_1's l2: 0.716043
[8]	training's l2: 0.721568	valid_1's l2: 0.681508
[9]	training's l2: 0.689113	valid_1's l2: 0.650425
[10]	training's l2: 0.659591	valid_1's l2: 0.622307
[11]	training's l2: 0.632947	valid_1's l2: 0.596708
[12]	training's l2: 0.608767	valid_1's l2: 0.573589
[13]	training's l2: 0.586652	valid_1's l2: 0.552765
[14]	training's l2: 0.566668	valid_1's l2: 0.533752
[15]	training's l2: 0.549637	valid_1's l2: 0.517416
[16]	training's l2: 0.533038	valid_1's l2: 0.501738
[17]	training's l2: 0.518933	valid_1's l2: 0.48826
[18]	training's l2: 0.505185	valid_1's l2: 0.475219
[19]	training's 

 94%|█████████▍| 15/16 [25:01<01:40, 100.06s/it]

[1]	training's l2: 0.955334	valid_1's l2: 0.937427
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.901319	valid_1's l2: 0.883671
[3]	training's l2: 0.852431	valid_1's l2: 0.835249
[4]	training's l2: 0.808292	valid_1's l2: 0.791357
[5]	training's l2: 0.768349	valid_1's l2: 0.751722
[6]	training's l2: 0.733286	valid_1's l2: 0.716865
[7]	training's l2: 0.7004	valid_1's l2: 0.684522
[8]	training's l2: 0.67073	valid_1's l2: 0.655358
[9]	training's l2: 0.643739	valid_1's l2: 0.628642
[10]	training's l2: 0.619304	valid_1's l2: 0.604571
[11]	training's l2: 0.597249	valid_1's l2: 0.582756
[12]	training's l2: 0.57731	valid_1's l2: 0.563142
[13]	training's l2: 0.55916	valid_1's l2: 0.545354
[14]	training's l2: 0.542764	valid_1's l2: 0.52915
[15]	training's l2: 0.527899	valid_1's l2: 0.514536
[16]	training's l2: 0.51437	valid_1's l2: 0.501214
[17]	training's l2: 0.502513	valid_1's l2: 0.489462
[18]	training's l2: 0.491401	valid_1's l2: 0.478553
[19]	training's l2

100%|██████████| 16/16 [26:37<00:00, 98.86s/it] 


#### Generate submission

In [82]:
mse = mean_squared_error(y_val, np.array(val_pred).transpose())
mse

0.34995690601359186

In [83]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_train.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)

Making submission...


In [85]:
153/1674

0.0913978494623656

In [86]:
mse = mean_squared_error(y_val, np.array(val_pred).transpose())

mlflow.set_experiment('grocery forecasting')
with mlflow.start_run(run_name='lgbm'):
    mlflow.log_param('model', 'lgbm')
    mlflow.log_param('train starts', train_start)
    mlflow.log_params(params)
    mlflow.log_param('lagging', LAG_DICT.values())
    mlflow.log_param('slidingWindows', SLIDING_DICT.values())
    mlflow.log_param('item_info', 'Yes')
    mlflow.log_param('store_info', 'Yes')
    mlflow.log_param('transactions', 'Yes')
    mlflow.log_param('private score', 0.52153)
    mlflow.log_param('private rank', '12%')
    mlflow.log_param('public score', 0.51516)

    mlflow.log_metric('mse', mse)
    
print("Validation mse:", mse)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/py4j/java_collections.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import (


Validation mse: 0.34995690601359186


Making submission...
